In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
import warnings
import gc
warnings.filterwarnings("ignore")

In [ ]:
def json_to_df(df, column):
    num_rows = len(df)
    data_list = []
    for row in tqdm(range(num_rows)):

        json_data = df.iloc[row][column]
        if str(json_data) != "nan":
            data = pd.read_json(json_data)
            data_list.append(data)
    all_data = pd.concat(data_list, axis = 0)
    
    return all_data

In [ ]:
%%time
file_path = '/kaggle/input/mlb-player-digital-engagement-forecasting/'
# train = pd.read_csv(file_path + 'train.csv')
test = pd.read_csv(file_path + 'example_test.csv')
submit = pd.read_csv(file_path + 'example_sample_submission.csv')
player_engagement = pd.read_pickle('/kaggle/input/mlb-player-engagement/train_player_engagement.pkl')

In [ ]:
# player_engagement = json_to_df(train, 'nextDayPlayerEngagement')
# player_engagement.insert(0, 'date', pd.to_datetime(player_engagement['engagementMetricsDate'])-\
#                                                    timedelta(days=1))
# player_engagement['engagementMetricsDate'] = pd.to_datetime(player_engagement['engagementMetricsDate'])
# player_engagement.reset_index(drop=True, inplace=True)
# player_engagement.engagementMetricsDate = player_engagement.engagementMetricsDate.astype(str)
# year = list(map(lambda x: x.split('-')[0] ,player_engagement.engagementMetricsDate))
# month = list(map(lambda x: x.split('-')[1] ,player_engagement.engagementMetricsDate))
# day = list(map(lambda x: x.split('-')[2] ,player_engagement.engagementMetricsDate))
# player_engagement['date_'] = list(map(lambda x,y,z: x+y+z,year,month,day ))
# player_engagement['date_playerId'] = list(map(lambda x,y: x+'_'+y,player_engagement.date_,player_engagement.playerId.astype(str)))
# player_engagement.drop('date_',axis=1,inplace=True)

In [ ]:
player_engagement['year'] = player_engagement.engagementMetricsDate.astype('datetime64').dt.year
player_engagement['month'] = player_engagement.engagementMetricsDate.astype('datetime64').dt.month
player_engagement['day'] = player_engagement.engagementMetricsDate.astype('datetime64').dt.day

In [ ]:
valid = player_engagement[(player_engagement.year==2021)&(player_engagement.month>=4)]
train = player_engagement.drop(valid.index)
valid.reset_index(drop=True,inplace=True)

In [ ]:
target1 = train.groupby(['playerId','year'])['target1'].agg(target1_mean='mean',
                                                                                        target1_std='std',
                                                                                                 target1_median='median',
                                                                                                 target1_max='max',
                                                                                                 target1_sum='sum',
                                                                                                 target1_min='min').reset_index()
target2 = train.groupby(['playerId','year'])['target2'].agg(target2_mean='mean',
                                                                                        target2_std='std',
                                                                                                 target2_median='median',
                                                                                                 target2_max='max',
                                                                                                 target2_sum='sum',
                                                                                                 target2_min='min').reset_index()
target3 = train.groupby(['playerId','year'])['target3'].agg(target3_mean='mean',
                                                                                        target3_std='std',
                                                                                                 target3_median='median',
                                                                                                 target3_max='max',
                                                                                                 target3_sum='sum',
                                                                                                 target3_min='min').reset_index()
target4 = train.groupby(['playerId','year'])['target4'].agg(target4_mean='mean',
                                                                                        target4_std='std',
                                                                                                 target4_median='median',
                                                                                                 target4_sum='sum',
                                                                                                 target4_max='max',
                                                                                                 target4_min='min').reset_index()

In [ ]:
train_data = train[['playerId','year','month','day']]
train_label = train[['target1','target2','target3','target4']]
for target in [target1,target2,target3,target4]:
        train_data = pd.merge(train_data,target,on=['year','playerId'],how='left')
        
test_data = valid[['playerId','year','month','day']]
test_label = valid[['target1','target2','target3','target4']]
for target in [target1,target2,target3,target4]:
        test_data = pd.merge(test_data,target,on=['year','playerId'],how='left') 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor

ALL_LOSS = 0
NUM_TREE = []
model = LGBMRegressor(n_estimators=1000,random_state=2020)
for target in ['target1','target2','target3','target4']:
    mae_loss = 0
    model.fit(train_data,train_label[target],eval_set=(test_data,test_label[target]),early_stopping_rounds=50,eval_metric='mae',verbose=False,
             categorical_feature=['playerId'])
    mae_loss = mean_absolute_error(model.predict(test_data),test_label[target])
        
    ALL_LOSS += mae_loss
    NUM_TREE.append(model.best_iteration_)
    print('mae_loss:',mae_loss)
print('finally loss:',ALL_LOSS/4)

In [ ]:
target1 = player_engagement.groupby(['playerId','year'])['target1'].agg(target1_mean='mean',target1_std='std',
                                                            target1_median='median',target1_max='max',
                                                            target1_sum='sum',target1_min='min').reset_index()
target2 = player_engagement.groupby(['playerId','year'])['target2'].agg(target2_mean='mean',target2_std='std',
                                                            target2_median='median',target2_max='max',
                                                            target2_sum='sum',target2_min='min').reset_index()
target3 = player_engagement.groupby(['playerId','year'])['target3'].agg(target3_mean='mean',target3_std='std',
                                                            target3_median='median',target3_max='max',
                                                            target3_sum='sum',target3_min='min').reset_index()
target4 = player_engagement.groupby(['playerId','year'])['target4'].agg(target4_mean='mean',target4_std='std',
                                                            target4_median='median',target4_max='max',
                                                            target4_sum='sum',target4_min='min').reset_index()
data = player_engagement[['playerId','year','month','day']]
label = player_engagement[['target1','target2','target3','target4']]
for target in [target1,target2,target3,target4]:
        data = pd.merge(data,target,on=['year','playerId'],how='left')

In [ ]:
model1 = LGBMRegressor(n_estimators=NUM_TREE[0],random_state=2020)
model2 = LGBMRegressor(n_estimators=NUM_TREE[1],random_state=2020)
model3 = LGBMRegressor(n_estimators=NUM_TREE[2],random_state=2020)
model4 = LGBMRegressor(n_estimators=NUM_TREE[3],random_state=2020)
model1.fit(data,label['target1'],eval_set=(data,label['target1']),eval_metric='mae',verbose=False,
             categorical_feature=['playerId'])
model2.fit(data,label['target2'],eval_set=(data,label['target2']),eval_metric='mae',verbose=False,
             categorical_feature=['playerId'])
model3.fit(data,label['target3'],eval_set=(data,label['target3']),eval_metric='mae',verbose=False,
             categorical_feature=['playerId'])
model4.fit(data,label['target4'],eval_set=(data,label['target4']),eval_metric='mae',verbose=False,
             categorical_feature=['playerId'])

In [ ]:
del data,label,train_data,train_label,test_data,test_label,train,player_engagement
gc.collect()

In [ ]:
import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    submit = pd.DataFrame()
    submit['playerId'] = sample_prediction_df['date_playerId'].str.split('_',expand=True)[1].astype('int64')
    submit['date'] = sample_prediction_df['date_playerId'].str.split('_',expand=True)[0].astype('int64')    
    submit['year'] = submit.date//10000
    submit['month'] = submit.date//100%100
    submit['day'] = submit.date%100
    for target in [target1,target2,target3,target4]:
        submit = pd.merge(submit,target,on=['year','playerId'],how='left')
    submit = submit.drop('date',axis=1)
    sample_prediction_df['target1'] = np.clip(model1.predict(submit),0,100) #make predictions here
    sample_prediction_df['target2'] = np.clip(model2.predict(submit),0,100)
    sample_prediction_df['target3'] = np.clip(model3.predict(submit),0,100)
    sample_prediction_df['target4'] = np.clip(model4.predict(submit),0,100)
    sample_prediction_df = sample_prediction_df.fillna(0)
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    print(sample_prediction_df)
    env.predict(sample_prediction_df)